### This Notebook can be used to compute the flux of MCP from meson decays in cosmic-showers

In [1]:
#--- Import relevant scripts used for the calculation of the MCP flux 

from Read_Meson import *
from Meson_Br import *
from Meson_Functions import *

import os
main = os.path.dirname(os.getcwd())

Read_Meson.py called! 
Meson_Br.py called! 
Meson_Functions.py called! 


In [2]:
from Repo_Path import *

C:\Users\vicmual\Desktop\Repository


In [3]:
#--- Define object to integrate
def Integrando_LogEp(LogEp,x,mMCP,Edau,idxc):
    pi0=2*np.exp(LogEp)*float(arr_pi0_X[idxc](x,np.exp(LogEp)))*dP_dE_3B(np.exp(LogEp), Edau, 2*mMCP,m_pi0)*arr_rho_X[idxc](x)**-1*decay_length_pi0(np.exp(LogEp))**-1
    return pi0

#--- Production Profile of MCP
def MCP_dLogXdE(LogX,e):
            return np.exp(LogX)*float(interpol_MCP_dX(np.exp(LogX),e))


In [4]:
#--- Set the mass and epsilon2 values for the computation:
mMCP_vec=[0.05]
eps2_vec=[1e-2] 

#--- Azimuthal contribution
ang_factor=2*PI

#--- Set energy grid for MCP flux
e_vec=np.unique(E_d)
e_vec_dau=np.logspace(np.log10(e_vec[0]),np.log10(e_vec[60]))

In [5]:
#--- Compute the flux of MCP at the surface of the earth:
dFlx_dE=[]
for mMCP_val in mMCP_vec:
    dFlx_dE_cos=[]
    for ic, cos in enumerate(cos_unique):
        int_val=[]
        x_val=[]
        e_val=[]   
        #--- ***small adjustment to avoid interpolation issues
        if cos==0.8:
            x_vec=np.logspace(np.log10(min(slant[-2])+0.001), np.log10(max(slant[-2])))
        elif cos==1.0:
            x_vec=np.logspace(np.log10(min(slant[-1])+0.01), np.log10(max(slant[-1])))
        else:    
            x_vec=np.logspace(np.log10(min(slant[ic])), np.log10(max(slant[ic])))
        #--- ***
        for X in x_vec:
            for E in e_vec_dau:
                I=integrate.quad(Integrando_LogEp,np.log(Epar_min(E,2*mMCP_val,m_pi0)),np.log(Epar_max(E,2*mMCP_val,m_pi0)),args=(X,mMCP_val,E,ic),epsabs= 1e-10, epsrel= 1e-10, limit=300)
                int_val.append(I[0])
                e_val.append(E)
                x_val.append(X)

        df_X = pd.DataFrame.from_dict({'X1':x_val, 'X2':e_val, 'F_X1X2': int_val})
        interpol_MCP_dX= interpolation_2D(df_X)

        dphi_dE=[]
        for E in e_vec_dau:
            I=integrate.quad(MCP_dLogXdE,np.log(min(x_vec)),np.log(max(x_vec)),args=(E),epsabs= 1e-10, epsrel= 1e-10, limit=300)
            dphi_dE.append(I[0]*ang_factor)
        dFlx_dE_cos.append(dphi_dE)
    dFlx_dE.append(dFlx_dE_cos)

<ipython-input-5-541d62c00fb2>:29: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  I=integrate.quad(MCP_dLogXdE,np.log(min(x_vec)),np.log(max(x_vec)),args=(E),epsabs= 1e-10, epsrel= 1e-10, limit=300)


In [6]:
#--- Multiply by Br(M-> MCP) to get the flux

def MCP_flux(Ei,idxc,idxm):
    E_val=Ei
    mcp_flx_val= dFlx_dE[idxm][idxc]
    mcp_flx=interpolate.interp1d(e_vec_dau,mcp_flx_val)
    mcp_surf=mcp_flx(E_val)
    return mcp_surf

eps2_dat=[]
m_dat=[]
e_dat=[]
cos_dat=[]
flx_dat=[]
e_delta=[]
for ide,eps2_val in enumerate(eps2_vec):
    for idm, mMCP_val in enumerate(mMCP_vec):
        for idc, cos in enumerate(cos_unique):
            for e in e_vec_dau: 
                eps2_dat.append(eps2_val)
                m_dat.append(mMCP_val)
                cos_dat.append(cos)
                flx_dat.append(Br_pi0_mCP(eps2_val,mMCP_val)*MCP_flux(e,idc,idm) )
                e_dat.append(e)

In [7]:
#--- Create data frame and save this in the 'Data/MCP_flux' folder
df=pd.DataFrame.from_dict({'dphi_dEdCth(1/GeV/s/cm2)':flx_dat,'Energy (GeV)':e_dat,'cosTh':cos_dat ,'m(GeV)':m_dat,'eps2':eps2_dat })
path_to_data=repo_path+'\Data\MCP_flux'
df.to_csv (path_to_data+r'\MCP_FROM_PI0_SURFACE.csv', index = None, header=True) 
df

,dphi_dEdCth(1/GeV/s/cm2),Energy (GeV),cosTh,m(GeV),eps2
0,5.839666e-08,1.349700e-01,0.0,0.05,0.01
1,2.848004e-08,1.936911e-01,0.0,0.05,0.01
2,1.801280e-08,2.779598e-01,0.0,0.05,0.01
3,1.205648e-08,3.988910e-01,0.0,0.05,0.01
4,8.248841e-09,5.724356e-01,0.0,0.05,0.01
...,...,...,...,...,...
295,2.394320e-25,1.547008e+06,1.0,0.05,0.01
296,6.787681e-26,2.220060e+06,1.0,0.05,0.01
297,1.904236e-26,3.185937e+06,1.0,0.05,0.01
298,5.430784e-27,4.572034e+06,1.0,0.05,0.01
